### Блок 2. Аналитические задачи

#### Секция 1. Анализ клиентов

**1. Сегментация по доходу:** Посчитайте средний годовой личный доход клиентов (YearlyIncome) в разбивке по роду деятельности (Occupation). Итоговая таблица должна содержать следующие поля: occupation, number_of_customers, avg_income.

In [ ]:
select 
    occupation,
    count(*) as number_of_customers,
    round(avg(yearly_income), 2) as avg_income
from adv_works.customers
group by 
	occupation
order by 
	avg_income desc;

**2. Семейный профиль:** Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: has_children (где 1 означает - имеет детей и 0 - не имеет детей), pct_of_customer_base.

In [ ]:
select 
    has_children,
    round(100.0 * count(*) / total.count, 2) as pct_of_customer_base
from (
    select 
        case 
            when number_of_children > 0 then 1
            else 0
        end as has_children
    from adv_works.customers
) as subquery,
(
    select count(*) as count from adv_works.customers
) as total
group by 
	has_children, total.count;

**3. Высокодоходные клиенты:** Сформируйте список топ 10 клиентов с наибольшей суммой покупок (поле SalesAmount). Итоговая таблица должна содержать следующие поля: customer_key, customer_name, total_purchase.

In [ ]:
select 
    c.customer_key,
    c.full_name,
    round(sum(s.sales_amount), 2) as total_purchase
from adv_works.customers c
join adv_works.sales s 
	on c.customer_key = s.customer_key
group by 
	c.customer_key, c.full_name
order by 
	total_purchase desc
limit 10;

**4. Влияние семейного положения:** Посчитайте среднюю сумму продаж в разбивке по семейному положению клиентов (MaritalStatus) и определите насколько сильно различаются средние суммы между двумя группами. Итоговая таблица должна содержать следующие поля: year, marital_status, avg_sales_amount.

In [ ]:
select 
    extract(year from s.order_date) as year,
    c.marital_status,
    round(avg(s.sales_amount), 2) as avg_sales_amount
from adv_works.customers c
join adv_works.sales s 
	on c.customer_key = s.customer_key
group by year, c.marital_status
order by year, avg_sales_amount desc;

#### Секция 2. Анализ продаж

**1. Ежемесячные продажи:** Создайте отчёт продаж по месяцам за последние 2 года (2003, 2004). Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, sales_count (количество продаж), sales_amount.

In [ ]:
select 
    extract(year from s.order_date) as year,
    to_char(s.order_date, 'YYYYMM') as monthkey,
    to_char(s.order_date, 'Month') as month_name,
    count(*) as sales_count,
    round(sum(s.sales_amount), 2) as sales_amount
from adv_works.sales s
where extract(year from s.order_date) in (2001, 2002, 2003, 2004)
group by year, monthkey, month_name
order by year, monthkey;

**2.Продажи по регионам:** Посчитайте сумму продаж в разбивке по регионам. Итоговая таблица должна содержать следующие поля: region, sales_count, sales_amount.

In [ ]:
select 
    t.region,
    count(*) as sales_count,
    round(sum(s.sales_amount), 2) as sales_amount
from adv_works.sales s
join adv_works.territory t on s.territory_key = t.territory_key
group by t.region
order by sales_amount desc;

#### Секция 3. Анализ продуктов

**1. Доля продаж:** Посчитайте какую долю от общих продаж составляет каждая категория продуктов. Итоговая таблица должна содержать следующие поля: year, product_key, product_category_key, english_product_category_name, sales_amount, pct_of_total_sales.

In [ ]:
select 
    extract(year from s.order_date) as year,
    p.product_key,
    pc.category_key as product_category_key,
    pc.category_name,
    round(sum(s.sales_amount), 2) as sales_amount,
    round(100.0 * sum(s.sales_amount) / total.total_sales, 2) as pct_of_total_sales
from adv_works.sales s
join adv_works.products p on s.product_key = p.product_key
join adv_works.product_subcategory psc on p.subcategory_key = psc.subcategory_key
join adv_works.product_category pc on psc.category_key = pc.category_key
cross join (
    select sum(sales_amount) as total_sales from adv_works.sales
) as total
group by year, p.product_key, pc.category_key, pc.category_name, total.total_sales
order by year, pct_of_total_sales desc;

**2. Самые продаваемые продукты:** Определите топ 5 продуктов с наибольшей суммой продаж. Итоговая таблица должна содержать следующие поля: product_key, product_name, english_product_category_name, sales_amount

In [ ]:
select 
    p.product_key,
    p.product_name,
    pc.category_name,
    round(sum(s.sales_amount), 2) as sales_amount
from adv_works.sales s
join adv_works.products p on s.product_key = p.product_key
join adv_works.product_subcategory psc on p.subcategory_key = psc.subcategory_key
join adv_works.product_category pc on psc.category_key = pc.category_key
group by p.product_key, p.product_name, pc.category_name
order by sales_amount desc
limit 5;

**3. Маржа от продаж:** Посчитайте разницу между суммой продаж (SalesAmount) за минусом себестоимости (TotalProductCost), налогов (*TaxAmt) и расходов на доставку (Freight) по каждому продукту в разбивке по годам и месяцам. Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, product_key, product_name, sales_amount, total_product_cost, tax_amt, freight, margin, margin_pct (маржа как процент от суммы продаж).

In [ ]:
select 
    extract(year from s.order_date) as year,
    to_char(s.order_date, 'YYYYMM') as monthkey,
    to_char(s.order_date, 'Month') as month_name,
    p.product_key,
    p.product_name,
    round(sum(s.sales_amount), 2) as sales_amount,
    round(sum(s.total_product_cost), 2) as total_product_cost,
    round(sum(s.tax_amount), 2) as tax_amt,
    round(sum(s.freight), 2) as freight,
    round(sum(
    	s.sales_amount - s.total_product_cost - s.tax_amount - s.freight
    	), 2) as margin,
    round(100.0 * sum(
    				s.sales_amount - s.total_product_cost - s.tax_amount - s.freight
    				) / sum(
    				s.sales_amount
    				), 2) as margin_pct
from adv_works.sales s
join adv_works.products p on s.product_key = p.product_key
group by year, monthkey, month_name, p.product_key, p.product_name
order by year, monthkey, margin desc;

#### Секция 4. Анализ трендов

**1. Квартальный рост:** Посчитайте сумму продаж за каждый квартал и их процентное изменение по топ 2 наиболее продаваемым категориям. Итоговая таблица должна содержать следующие поля: year, quarter_id, product_category_key, english_product_category_name, quarter_sales_amount, quarter_over_quarter_growth_pct.

**2. Сравнение будних и выходных (суббота, воскресенье) дней:** Посчитайте продажи в разбивке по годам и дням недели. Определите в какие дни в среднем сумма продаж больше. Определите является ли сумма продаж больше в будние или выходные дни. Итоговая таблица должна содержать следующие поля: year, day_name, is_weekend (где 1 означает выходной а 0 будний день), sales_amount.

## Case Study по модулю SQL
### Назирзода Рахимджон 04-03-2025

### Блок 1. Создание схемы и таблиц

**1.В базе данных создайте новую схему adv_works.**

**2.На основе данных из файла создайте в схеме adv_works таблицы и соедините их между собою по основным и внешним ключам.**

In [4]:
import json
import pandas as pd
from sqlalchemy import text
#from sqlalchemy.engine.url import URL

from connect import set_connection

ModuleNotFoundError: No module named 'connect'

In [37]:
SCHEMA = "adv_works"

In [35]:
def create_tables():
    try:
        
        with open('queries/cas_study_on_sql.sql', encoding='utf-8') as f:
            tables_query = f.read()
        
        # создаём схему и таблицы
        with set_connection() as psg:
            psg.execute(text(tables_query))
            psg.commit()
        
        print("Tables created successfully")
    except Exception as e:
        print(e)

In [36]:
create_tables()

Tables created successfully


In [38]:
def read_xl(sheet_name, columns_dict):
    temp_df = pd.read_excel(
        'adventure_works.xlsx',
        sheet_name=sheet_name,
        usecols=columns_dict.keys()
    ).rename(columns=columns_dict)
    return temp_df

In [39]:
with open('columns.json') as f:
    tables_dict = json.load(f) 

In [40]:
def insert_to_db(temp_df, tbl_name):
    with set_connection() as psg:
        temp_df.to_sql(
            schema=SCHEMA,
            name=tbl_name,
            con=psg,
            index=False,
            if_exists='append'
        )

In [41]:
def etl(sheet_name, columns_dict, tbl_name):
    print(f"inserting data to {tbl_name}...")
    temp_df = read_xl(sheet_name, columns_dict)
    insert_to_db(temp_df, tbl_name)

In [42]:
for k, v in tables_dict.items():
    etl(k, v["columns"], v["table_name"])

inserting data to customers...
inserting data to territory...
inserting data to product_category...
inserting data to product_subcategory...
inserting data to products...
inserting data to sales...
